In [ ]:
from utils.helpers import get_api_key
import os

os.environ['OPENAI_API_KEY'] = get_api_key()


# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-4-0125-preview"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import openai
import re
import httpx
import os
from openai import OpenAI

In [ ]:
# What is temparature passed to model parameter
# temperature = 0.0, => deterministic output
# temperature = 0.1, => low temperature, more deterministic output
# temperature = 0.9, => high temperature, more random output

In [ ]:
client = OpenAI()

In [ ]:
chat_completion = client.chat.completions.create(
    model=llm_model,
    messages=[{"role":"user", "content":"Hello openai"}]
)

In [ ]:
chat_completion.choices[0].message.content

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message:str):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        response = client.chat.completions.create(
            model=llm_model,
            messages=self.messages,
            temperature=0.0
        )
        return response.choices[0].message.content.strip()

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [ ]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [ ]:
average_dog_weight('Toy Poodle')

In [ ]:
calculate('2*2')

In [ ]:
abot = Agent(system=prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")

In [ ]:
print(result)

In [ ]:
result = average_dog_weight("Toy Poodle")

In [ ]:
result

In [ ]:
next_prompt = "Observation: {}".format(result)

In [ ]:
abot(next_prompt)

In [ ]:
abot.messages

In [ ]:
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
action_re = re.compile('^Action: (\w+): (.*)$')

In [ ]:
def query(question, max_turns:int = 5):
    i = 0
    bot = Agent(system=prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        thought = bot(next_prompt)
        print("Thought: {}".format(thought))
        actions = [
            action_re.match(line) for line in thought.splitlines() if action_re.match(line)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise ValueError(f"Unknown action: {action}")
            print(f"Action: {action} with input: {action_input}")
            observation = known_actions[action](action_input)
            print(f"Observation: {observation}")
            next_prompt = "Observation: {}".format(observation)
            
        else:
            print("No actions found in thought, returning answer")
            return thought

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)